In [1]:
!nvidia-smi

Tue Jun 21 15:53:49 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 462.30       Driver Version: 462.30       CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  GeForce GTX 165... WDDM  | 00000000:01:00.0 Off |                  N/A |
| N/A   40C    P8     5W /  N/A |    134MiB /  4096MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## Library

In [6]:
import os
import re
import copy
import time
import json
import random
import numpy as np
import pandas as pd
from tqdm import tqdm
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder

from googletrans import Translator
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import tensor
from torch.utils.data import dataset, dataloader
from torch.nn.modules.loss import _WeightedLoss
from transformers import logging, AutoTokenizer, AutoModelForSequenceClassification, AutoModel

import warnings
warnings.filterwarnings(action='ignore')

In [7]:
# hyperparameters

seed = 10
num_labels = 3
KorNLU_num = 50000
n_fold = 5
epochs = 100
patience = 5
max_seq_len = 103
batch_size = 128
weight_decay = 0.0001
drop_out = 0.1
label_smooting = 0.2
lr = 5e-5
max_grad_norm = 10

device = '0,1,2,3'
text = 'text'
model_name = 'klue/roberta-large'
library_name = 'AutoModelForSequenceClassification'
Model_library = eval(library_name)

model_save_folder = 'results/'
model_save = '{}_{}_{}_{}_{}_{}_{}_{}_{}_fold'.format(text,
                                                    KorNLU_num,
                                                    lr,
                                                    max_seq_len,
                                                    n_fold,
                                                    batch_size,
                                                    weight_decay,
                                                    drop_out,
                                                    patience)

os.environ['PYTHONHASHSEED'] = str(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(seed)
random.seed(seed)

os.environ['CUDA_DEVICE_ORDER'] = 'PCI_BUS_ID'      # arrange GPU starting from 0
os.environ['CUDA_VISIBLE_DEVICES'] = device

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('device: %s' %device)
if (device.type == 'cuda') or (torch.cuda.device_count() > 1):
    print('GPU activate --> Count of using GPUs: %s' % torch.cuda.device_count())

device: cpu


## data load

In [10]:
# KLUE data load

json_train_path = 'klue-nli-v1.1_train.json'
json_test_path = 'klue-nli-v1.1_dev.json'

with open(json_train_path, 'r', encoding='utf-8') as f:
    json_train = json.load(f)
with open(json_test_path, 'r', encoding='utf-8') as f:
    json_test = json.load(f)

json_train_df = pd.DataFrame(json_train)[['premise', 'hypothesis', 'gold_label']]
json_test_df = pd.DataFrame(json_test)[['premise', 'hypothesis', 'gold_label']]
json_train_df.rename(columns={'gold_label':'label'}, inplace=True)
json_test_df.rename(columns={'gold_label':'label'}, inplace=True)
df1 = pd.concat([json_train_df, json_test_df]).reset_index(drop=True)

df_grp1 = df1.groupby(df1.columns.tolist()) # 전체 열 비교
df_dict1 = df_grp1.groups # 딕셔너리로 만들기
idx_T1 = [x[0] for x in df_dict1.values() if len(x) == 1] # 중복 없는 인덱스 검토
idx_F1 = [x[0] for x in df_dict1.values() if not len(x) == 1] # 중복 있는 인덱스 검토
df_concat1 = pd.concat([df1.loc[idx_T1, :], df1.loc[idx_F1, :]])
df_concat1 = df_concat1.dropna(how='any') # null값이 존재하는 행 제거
df_concat1 = df_concat1.reset_index(drop=True)

In [16]:
df_concat1

,premise,hypothesis,label
0,100분간 잘껄 그래도 소닉붐땜에 2점준다,100분간 자는게 더 나았을 것 같다.,neutral
1,100분간 잘껄 그래도 소닉붐땜에 2점준다,100분간 잤다.,contradiction
2,100분간 잘껄 그래도 소닉붐땜에 2점준다,소닉붐이 정말 멋있었다.,neutral
3,101빌딩 근처에 나름 즐길거리가 많습니다.,101빌딩 근처에서 즐길거리 찾기는 어렵습니다.,contradiction
4,101빌딩 근처에 나름 즐길거리가 많습니다.,101빌딩 부근에서는 여러가지를 즐길수 있습니다.,entailment
...,...,...,...
27991,힛걸 진심 최고다 그 어떤 히어로보다 멋지다,힛걸 그 어떤 히어로보다 별로다.,contradiction
27992,힛걸 진심 최고다 그 어떤 히어로보다 멋지다,힛걸 액션 장면 진심 그 어떤 히어로보다 멋지다.,neutral
27993,힛걸 진심 최고다 그 어떤 히어로보다 멋지다,힛걸 진심 최고로 멋지다.,entailment
27994,"3층 포루는 정면 2칸, 측면 2칸의 팔작 기와지붕으로 벽면 위쪽의 판문에는 전안이...",전안은 벽면 위쪽의 판문에 설치되어 있다.,entailment


In [27]:
# KorNLU data load

multinli_path = 'https://github.com/kakaobrain/KorNLUDatasets/raw/master/KorNLI/multinli.train.ko.tsv'
snli_path = 'https://github.com/kakaobrain/KorNLUDatasets/raw/master/KorNLI/snli_1.0_train.ko.tsv'
xnli_dev_path = 'https://github.com/kakaobrain/KorNLUDatasets/raw/master/KorNLI/xnli.dev.ko.tsv'
xnli_test_path = 'https://github.com/kakaobrain/KorNLUDatasets/raw/master/KorNLI/xnli.test.ko.tsv'

multinli_data = pd.read_csv(multinli_path, sep='\t', error_bad_lines=False).sample(KorNLU_num)
snli_data = pd.read_csv(snli_path, delimiter='\t', error_bad_lines=False).sample(KorNLU_num)
xnli_dev_data = pd.read_csv(xnli_dev_path, delimiter='\t', error_bad_lines=False)
xnli_test_data  = pd.read_csv(xnli_test_path, delimiter='\t', error_bad_lines=False)

multinli_data.rename(columns = {'gold_label' : 'label', 'sentence1' : 'premise', 'sentence2' : 'hypothesis'}, inplace = True)
snli_data.rename(columns = {'gold_label' : 'label', 'sentence1' : 'premise', 'sentence2' : 'hypothesis'}, inplace = True)
xnli_test_data.rename(columns = {'gold_label' : 'label', 'sentence1' : 'premise', 'sentence2' : 'hypothesis'}, inplace = True)
xnli_dev_data.rename(columns = {'gold_label' : 'label', 'sentence1' : 'premise', 'sentence2' : 'hypothesis'}, inplace = True)    
df2 = pd.concat([multinli_data,snli_data, xnli_test_data,xnli_dev_data]).reset_index(drop=True)

df_grp2 = df2.groupby(df2.columns.tolist()) # 전체 열 비교
df_dict2 = df_grp2.groups # 딕셔너리로 만들기 
idx_T2 = [x[0] for x in df_dict2.values() if len(x) == 1] # 중복X 인덱스 검토
idx_F2 = [x[0] for x in df_dict2.values() if not len(x) == 1] # 중복O 인덱스 검토
df_concat2 = pd.concat([df2.loc[idx_T2,:], df2.loc[idx_F2,:]])
df_concat2 = df_concat2.dropna(how='any') # Null 값이 존재하는 행 제거

df_concat2['premise'] = df_concat2['premise'].str.replace('[^ㄱ-ㅎㅏ-ㅣ가-힣 0-9 a-z A-Z]', '', regex=True)
df_concat2['hypothesis'] = df_concat2['hypothesis'].str.replace('[^ㄱ-ㅎㅏ-ㅣ가-힣 0-9 a-z A-Z]', '', regex=True)

df_concat2 = df_concat2[df_concat2['premise'].apply(lambda x: len(x) >= 19)] # 'premise' column에 글자수 19 이하인 데이터 제거
df_concat2 = df_concat2[df_concat2['premise'].apply(lambda x: len(x) <= 90)] # 'premise' column에 글자수 90 이상인 데이터 제거
df_concat2 = df_concat2[df_concat2['hypothesis'].apply(lambda x: len(x) >= 5)]
df_concat2 = df_concat2[df_concat2['hypothesis'].apply(lambda x: len(x) <= 103)]
df_concat2 = df_concat2.reset_index(drop=True)

b'Skipping line 24426: expected 3 fields, saw 4\nSkipping line 156343: expected 3 fields, saw 4\nSkipping line 218766: expected 3 fields, saw 4\nSkipping line 232318: expected 3 fields, saw 4\nSkipping line 253493: expected 3 fields, saw 4\n'
b'Skipping line 265734: expected 3 fields, saw 4\nSkipping line 282588: expected 3 fields, saw 4\nSkipping line 350969: expected 3 fields, saw 4\n'


In [28]:
display(df_concat1); display(df_concat2)

,premise,hypothesis,label
0,100분간 잘껄 그래도 소닉붐땜에 2점준다,100분간 자는게 더 나았을 것 같다.,neutral
1,100분간 잘껄 그래도 소닉붐땜에 2점준다,100분간 잤다.,contradiction
2,100분간 잘껄 그래도 소닉붐땜에 2점준다,소닉붐이 정말 멋있었다.,neutral
3,101빌딩 근처에 나름 즐길거리가 많습니다.,101빌딩 근처에서 즐길거리 찾기는 어렵습니다.,contradiction
4,101빌딩 근처에 나름 즐길거리가 많습니다.,101빌딩 부근에서는 여러가지를 즐길수 있습니다.,entailment
...,...,...,...
27991,힛걸 진심 최고다 그 어떤 히어로보다 멋지다,힛걸 그 어떤 히어로보다 별로다.,contradiction
27992,힛걸 진심 최고다 그 어떤 히어로보다 멋지다,힛걸 액션 장면 진심 그 어떤 히어로보다 멋지다.,neutral
27993,힛걸 진심 최고다 그 어떤 히어로보다 멋지다,힛걸 진심 최고로 멋지다.,entailment
27994,"3층 포루는 정면 2칸, 측면 2칸의 팔작 기와지붕으로 벽면 위쪽의 판문에는 전안이...",전안은 벽면 위쪽의 판문에 설치되어 있다.,entailment


,premise,hypothesis,label
0,LSC 보조금 수령자가 매년 제공하는 총 인원은 CSRCase Service Re...,LSC 보조금은 CSR에 보고된 것보다 여러 배나 더 많은 서비스를 제공한다,entailment
1,점점 더 LSC 보조금 수령자는 저소득층에게 서비스를 제공하는 기관과 서비스 제공...,중요한 문지기는 사람들이 수입을 늘리기를 원하기 때문에 추천 네트워크의 LSC 보조...,neutral
2,지역 사회 법률 교육 지역 사회 기관에 대한 지원 및 추천과 같은 서비스가 LSC...,LSC는 저소득층 지역사회 구성원들을 돕는 사람들에게 보조금을 준다,entailment
3,2 그레이하운드 개가 트랙을 달리고 있습니다,개가 트랙을 뛰어다닌다,entailment
4,또한 동부와는 달리 거의 모든 사람들이 추잡할 정도로 어리기 때문에 이 IPO 머...,거의 모든 사람들이 추잡할 정도로 어리기 때문에 이러한 IPO 머리의 나이에 대한 ...,entailment
...,...,...,...
86182,힙합은 1990년대의 재즈다 분노 진정성 자본주의 유머의 혼합은 미국의 문화적 미학...,힙합은 1990년대의 재즈이다,entailment
86183,6명의 남자가 태양으로부터 눈을 가린다,남자들이 눈을 가리고 있다,entailment
86184,갈색 재킷을 입고 회색 지팡이를 들고 휠체어를 탄 남자가 덤불과 나무 앞에 앉아 있다,남자가 휠체어를 타고 있다,entailment
86185,밤에 나무 줄기에 파란 바퀴를 달고 자전거를 타는 사람,사람이 밤에 자전거를 타고 있다,entailment
